<a href="https://colab.research.google.com/github/bountyhunter12/Carbon-Aware-Computing/blob/main/Carbon_intensity_on_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Create a dictionary with the coordinates
coordinates = {
    "lat":34.00906474557528,
    "lon": -118.4984580927553
}

**Electricity Map's live carbon intensity endpoint**

In [4]:
# Build the url
url= f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

# Print the endpoint
print("Endpoint: " + str(url))

Endpoint: https://api.electricitymap.org/v3/carbon-intensity/latest?lat=34.00906474557528&lon=-118.4984580927553


In [5]:
import json

In [6]:
import requests

response = requests.get(
    f"https://api.electricitymaps.com/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}",
    headers={
        "auth-token": f"9DZxQaBNJ4VtyC94x2UR"
    }
)
print(response.json())

{'zone': 'US-CAL-LDWP', 'carbonIntensity': 411, 'datetime': '2025-08-02T08:00:00.000Z', 'updatedAt': '2025-08-02T08:43:14.798Z', 'createdAt': '2025-07-30T08:42:43.815Z', 'emissionFactorType': 'lifecycle', 'isEstimated': True, 'estimationMethod': 'TIME_SLICER_AVERAGE', 'temporalGranularity': 'hourly'}


**Use the live power breakdown endpoint.**

In [7]:
import requests

response = requests.get(
    f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}",
    headers={
        "auth-token": f"9DZxQaBNJ4VtyC94x2UR"
    }
)
power_breakdown = json.loads(response.content)

# Print the content
power_breakdown

{'zone': 'US-CAL-LDWP',
 'datetime': '2025-08-02T08:00:00.000Z',
 'updatedAt': '2025-08-02T08:43:14.798Z',
 'createdAt': '2025-07-30T08:42:43.815Z',
 'powerConsumptionBreakdown': {'nuclear': 136,
  'geothermal': 0,
  'biomass': 0,
  'coal': 372,
  'wind': 331,
  'solar': 0,
  'hydro': 643,
  'gas': 1458,
  'oil': 1,
  'unknown': 148,
  'hydro discharge': 66,
  'battery discharge': 12},
 'powerProductionBreakdown': {'nuclear': None,
  'geothermal': None,
  'biomass': None,
  'coal': 140,
  'wind': 116,
  'solar': 0,
  'hydro': 272,
  'gas': 1743,
  'oil': None,
  'unknown': 232,
  'hydro discharge': 115,
  'battery discharge': 0},
 'powerImportBreakdown': {'US-NW-BPAT': 1508,
  'US-NW-NEVP': 0,
  'US-NW-PACE': 459,
  'US-SW-AZPS': 277,
  'US-SW-WALC': 639,
  'US-CAL-CISO': 0},
 'powerExportBreakdown': {'US-NW-BPAT': 0,
  'US-NW-NEVP': 1259,
  'US-NW-PACE': 0,
  'US-SW-AZPS': 0,
  'US-SW-WALC': 0,
  'US-CAL-CISO': 1073},
 'fossilFreePercentage': 38,
 'renewablePercentage': 33,
 'powerCon

**Print some specific values.**

In [8]:
power_breakdown['renewablePercentage']

33

In [9]:
power_breakdown['fossilFreePercentage']

38

In [10]:
# Power Consumption Breakdown in MegaWatts
power_breakdown['powerConsumptionBreakdown']

{'nuclear': 136,
 'geothermal': 0,
 'biomass': 0,
 'coal': 372,
 'wind': 331,
 'solar': 0,
 'hydro': 643,
 'gas': 1458,
 'oil': 1,
 'unknown': 148,
 'hydro discharge': 66,
 'battery discharge': 12}

**some math to understand better the values above.**

In [11]:
import numpy as np

In [12]:
total_consumption = power_breakdown['powerConsumptionTotal']

In [13]:
total_consumption

3168

In [14]:
consumption_percent = {
    k: np.round((v/total_consumption) * 100)
    for k,v
    in power_breakdown['powerConsumptionBreakdown'].items()}
consumption_percent

{'nuclear': np.float64(4.0),
 'geothermal': np.float64(0.0),
 'biomass': np.float64(0.0),
 'coal': np.float64(12.0),
 'wind': np.float64(10.0),
 'solar': np.float64(0.0),
 'hydro': np.float64(20.0),
 'gas': np.float64(46.0),
 'oil': np.float64(0.0),
 'unknown': np.float64(5.0),
 'hydro discharge': np.float64(2.0),
 'battery discharge': np.float64(0.0)}

**Helper function for the power_stats**

In [15]:
def power_stats(lat, lon, api_key="9DZxQaBNJ4VtyC94x2UR"):
    coordinates = {"lat": lat, "lon": lon}

    url_intensity = f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_intensity = requests.get(url_intensity, headers={"auth-token": api_key})
    intensity = json.loads(request_intensity.content)

    url_breakdown = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_breakdown = requests.get(url_breakdown, headers={"auth-token": api_key})
    breakdown = json.loads(request_breakdown.content)

    # Debug print to inspect the full response
    print("BREAKDOWN RESPONSE:")
    print(json.dumps(breakdown, indent=2))

    # Optional: check for error
    if 'error' in breakdown:
        return intensity, {"error": breakdown['message']}

    # Safely access with .get() to prevent KeyErrors
    breakdown_abridged = {
        'renewablePercentage': breakdown.get('renewablePercentage', 'N/A'),
        'fossilFreePercentage': breakdown.get('fossilFreePercentage', 'N/A'),
        'powerConsumptionBreakdown': breakdown.get('powerConsumptionBreakdown', {}),
        'consumption_percent': {
            k: np.round((v / breakdown['powerConsumptionTotal']) * 100)
            for k, v in breakdown.get('powerConsumptionBreakdown', {}).items()
        } if breakdown.get('powerConsumptionTotal') else {}
    }

    return intensity, breakdown_abridged


In [20]:
# Coordinates from a landmark in Taiwan, shown by the instructor in the explanation
intensity, breakdown = power_stats(
    lat=34.06105197282242,
    lon=-118.4429445707998)


BREAKDOWN RESPONSE:
{
  "zone": "US-CAL-LDWP",
  "datetime": "2025-08-02T08:00:00.000Z",
  "updatedAt": "2025-08-02T08:43:14.798Z",
  "createdAt": "2025-07-30T08:42:43.815Z",
  "powerConsumptionBreakdown": {
    "nuclear": 136,
    "geothermal": 0,
    "biomass": 0,
    "coal": 372,
    "wind": 331,
    "solar": 0,
    "hydro": 643,
    "gas": 1458,
    "oil": 1,
    "unknown": 148,
    "hydro discharge": 66,
    "battery discharge": 12
  },
  "powerProductionBreakdown": {
    "nuclear": null,
    "geothermal": null,
    "biomass": null,
    "coal": 140,
    "wind": 116,
    "solar": 0,
    "hydro": 272,
    "gas": 1743,
    "oil": null,
    "unknown": 232,
    "hydro discharge": 115,
    "battery discharge": 0
  },
  "powerImportBreakdown": {
    "US-NW-BPAT": 1508,
    "US-NW-NEVP": 0,
    "US-NW-PACE": 459,
    "US-SW-AZPS": 277,
    "US-SW-WALC": 639,
    "US-CAL-CISO": 0
  },
  "powerExportBreakdown": {
    "US-NW-BPAT": 0,
    "US-NW-NEVP": 1259,
    "US-NW-PACE": 0,
    "US-SW-A

In [17]:
intensity

{'zone': 'US-CAL-LDWP',
 'carbonIntensity': 411,
 'datetime': '2025-08-02T08:00:00.000Z',
 'updatedAt': '2025-08-02T08:43:14.798Z',
 'createdAt': '2025-07-30T08:42:43.815Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE',
 'temporalGranularity': 'hourly'}

In [18]:
breakdown

{'renewablePercentage': 33,
 'fossilFreePercentage': 38,
 'powerConsumptionBreakdown': {'nuclear': 136,
  'geothermal': 0,
  'biomass': 0,
  'coal': 372,
  'wind': 331,
  'solar': 0,
  'hydro': 643,
  'gas': 1458,
  'oil': 1,
  'unknown': 148,
  'hydro discharge': 66,
  'battery discharge': 12},
 'consumption_percent': {'nuclear': np.float64(4.0),
  'geothermal': np.float64(0.0),
  'biomass': np.float64(0.0),
  'coal': np.float64(12.0),
  'wind': np.float64(10.0),
  'solar': np.float64(0.0),
  'hydro': np.float64(20.0),
  'gas': np.float64(46.0),
  'oil': np.float64(0.0),
  'unknown': np.float64(5.0),
  'hydro discharge': np.float64(2.0),
  'battery discharge': np.float64(0.0)}}